## Overview:
Group size 2

In [1]:
import numpy as np
import pandas as pd
from pandas import *
import sys
from collections import defaultdict
import xlrd
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import csv
import math
from datetime import datetime

In [14]:
# Indices
iset = range(0,41)
jset = {30,60,90,120,180,240,300,360}
demands = {30:2,60:7,90:6,120:8,180:7,240:5,300:2,360:3}
M = 100000
kset = range(1,19)
rset = range(1,7)
gset = [1,2,3]

# Groups are [1,2] [3,4] and [5,6]
f = {}
f[1] = [1,2,3,4,5,6]
f[2] = [7,8,9,10,11,12]
f[3] = [13,14,15,16,17,18]

# Busy times of each nurse
e = {}
for r in rset:
    e[r] = []
e[1] = [0,1,16,17,34,35,36,37,38,39,40]
e[2] = [0,1,2,3,18,19,36,37,38,39,40]
e[3] = [0,1,2,3,4,5,20,21,38,39,40]
e[4] = [0,1,2,3,4,5,20,21,38,39,40]
e[5] = [0,1,2,3,4,5,6,7,22,23,40]
e[6] = [0,1,2,3,4,5,6,7,22,23,40]

n={}
for i in iset:
    for g in gset:
        n[i] = {g:0}

for i in iset:
    for g in gset:
        if g == 1:
            if i in e[1]:
                if i in e[2]:
                    n[i][g] = 0
                else:
                    n[i][g] = 1
            elif i in e[2]:
                n[i][g] = 1
            else:
                n[i][g] = 2
        elif g == 2:
            if i in e[3]:
                if i in e[4]:
                    n[i][g] = 0
                else:
                    n[i][g] = 1
            elif i in e[4]:
                n[i][g] = 1
            else:
                n[i][g] = 2
        elif g == 3:
            if i in e[5]:
                if i in e[6]:
                    n[i][g] = 0
                else:
                    n[i][g] = 1
            elif i in e[6]:
                n[i][g] = 1
            else:
                n[i][g] = 2
        else:
            print("error")

In [15]:
t1 = datetime.now()
env = gp.Env(empty=True)
#env.setParam("OutputFlag",0)
env.start()
m = Model(name="Scheduling -- Group Size 2",env=env,)
adict = demands

# Decision Variables
t = m.addVars(iset, jset, vtype=GRB.INTEGER, name="t")
c = m.addVars(iset, jset, kset, vtype=GRB.BINARY, name="c")   # c=1 if appointment of length j starts at time i in chair k
u = m.addVars(iset, jset, kset, vtype=GRB.INTEGER, name="u")  # number of nurses actively needed at chair k at time i for j length appointments
b = m.addVars(iset, jset, kset, vtype=GRB.BINARY, name="b")   # b=1 if there is an appointment of length j scheduled in chair k at time i

# Violation Variables
vA = m.addVars(iset, vtype=GRB.INTEGER, name="violation A")
vB = m.addVars(iset, vtype=GRB.INTEGER, name="violation B")
vP = m.addVars(iset, vtype=GRB.INTEGER, name="violation pharmacy")
vD = m.addVars(iset, gset, vtype=GRB.INTEGER, name="violation D")            # violation if more than one nurse required
vS = m.addVars(jset, vtype=GRB.INTEGER, name="violation S")      # violation if an appointment is not scheduled

# Cannot schedule appointments that go past 5:00 PM (strict)
c2 = m.addConstrs(c[i,j,k] ==0 for j in jset for i in iset if i>(39-int(j/15)) for k in kset)

# Cannot schedule appointments before 7:30 AM (strict)
c3 = m.addConstr(gp.quicksum(c[0,j,k]+c[1,j,k] for j in jset for k in kset) ==0)

# Pharmacy can do 3 drugs per 30 minutes (add in violation)
c4a = m.addConstrs(gp.quicksum(c[i,j,k]+c[i-1,j,k] for j in jset for k in kset) - vP[i] <= 3 for i in iset if i>1)
c4b = m.addConstrs(gp.quicksum(c[i,j,k] for j in jset for k in kset) - vP[i] <= 3 for i in iset if i==0)

# Cumulative number of patients in system
c5a = m.addConstrs(t[i,j] == gp.quicksum(c[i-x,j,k] for k in kset for x in range(0,i+1)) for i in iset for j in jset if i+1 < int(j/15))
c5b = m.addConstrs(t[i,j] == gp.quicksum(c[i-x,j,k] for k in kset for x in range(0,int(j/15))) for i in iset for j in jset if i+1 >= int(j/15))

# Times when nurses are actively required at chairs
c6 = m.addConstrs(u[0,j,k] == 0 for j in jset for k in kset)
c7 = m.addConstrs(u[i,j,k] == c[i,j,k]+c[i-1,j,k] for j in jset for i in iset if i<math.floor(29*j/450) if i>0 for k in kset)
c8a = m.addConstrs(u[i,j,k] == c[i,j,k]+c[i-1,j,k]+c[i-math.floor(29*j/450),j,k] for j in jset if j != 30 for i in iset if i>=math.floor(29*j/450) if i>0 for k in kset)
c8b = m.addConstrs(u[i,j,k] == c[i,j,k]+c[i-1,j,k] for j in jset if j == 30 for i in iset if i>=math.floor(29*j/450) if i>0 for k in kset)

# Nurse busy times (when they cannot be actively required)
# Let g[r][i] be the number of nurses available in nurse r's group at time i
c9 = m.addConstrs(gp.quicksum(u[i,j,k] for j in jset for k in f[g])-vD[i,g] <= n[i][g] for g in gset for i in iset)

# Violation S: Meet demands
c11 = m.addConstrs(gp.quicksum(c[i,j,k] for i in iset for k in kset)+vS[j] == adict[j] for j in jset)

# Total number of appointments at any time cannot exceed number of chairs
c12 = m.addConstrs(gp.quicksum(t[i,j] for j in jset) <= len(kset) for i in iset)

# Violation A
c13 = m.addConstrs(gp.quicksum(u[i,j,k] for j in jset for k in kset) - gp.quicksum(n[i][g] for g in gset) <= vA[i] for i in iset)

# Violation B
c14 = m.addConstrs(gp.quicksum(t[i,j] for j in jset) <= gp.quicksum(n[i][g] for g in gset)*(3+vB[i]) for i in iset)

# Busy times of chairs
c15 = m.addConstrs(b[0,j,k] == 0 for j in jset for k in kset)
c16 = m.addConstrs(b[i,j,k] == gp.quicksum(c[i-m,j,k] for m in range(0,i+1)) for j in jset for i in iset if i<math.floor(29*j/450) if i>0 for k in kset)
c17 = m.addConstrs(b[i,j,k] == gp.quicksum(c[i-m,j,k] for m in range(0,int(j/15))) for j in jset for i in iset if i>=math.floor(29*j/450) if i>0 for k in kset)

# Maximum of one appointment may occur at a time at each chair
c18 = m.addConstrs(gp.quicksum(b[i,j,k] for j in jset) <= 1 for i in iset for k in kset)

# Objective: minimize all violations
obj1 = gp.quicksum(vA[i]+vB[i] for i in iset)
obj2 = gp.quicksum(vP[i] for i in iset)
obj3 = gp.quicksum(vD[i,g] for i in iset for g in gset)
obj4 = gp.quicksum(vS[j]*int(j/15) for j in jset)
obj = obj1 + obj2 + obj3 + obj4
m.setObjective(obj, GRB.MINIMIZE)

m.optimize()
t2 = datetime.now()

runtime = t2 - t1

var_names_violations = []
for var in m.getVars():
    if var.varName[0] == "v":
        if var.X != 0:
            var_names_violations.append(str(var.varName)+str(var.X))

var_names = []
for var in m.getVars():
    if var.X != 0:
        var_names.append(str(var.varName)+str(var.X))
        
var_names_start = []
for var in m.getVars():
    if var.varName[0] == "c":
        if var.X != 0:
            var_names_start.append(str(var.varName)+str(var.X))
            
print(var_names_violations)

Academic license - for non-commercial use only - expires 2023-09-13
Using license file C:\Users\lmdan\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 10187 rows, 12390 columns and 117538 nonzeros
Model fingerprint: 0x9cbcfa38
Variable types: 0 continuous, 12390 integer (7872 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+01]
Found heuristic solution: objective 408.0000000
Presolve removed 9339 rows and 9459 columns
Presolve time: 1.03s
Presolved: 848 rows, 2931 columns, 50115 nonzeros
Variable types: 0 continuous, 2931 integer (2568 binary)

Root relaxation: objective 3.300000e+01, 3136 iterations, 0.77 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Ti

In [9]:
runtime

datetime.timedelta(seconds=1, microseconds=368964)

In [16]:
var_names_violations

[]

In [19]:
var_names_start

['c[2,360,6]1.0',
 'c[4,120,1]1.0',
 'c[5,300,4]1.0',
 'c[6,180,5]1.0',
 'c[6,120,12]1.0',
 'c[7,240,10]1.0',
 'c[8,240,11]1.0',
 'c[8,120,3]1.0',
 'c[9,60,13]1.0',
 'c[10,240,15]1.0',
 'c[10,180,2]1.0',
 'c[11,360,7]1.0',
 'c[12,240,18]1.0',
 'c[12,180,9]1.0',
 'c[13,60,1]1.0',
 'c[15,360,17]1.0',
 'c[15,60,13]1.0',
 'c[16,60,8]1.0',
 'c[17,120,12]1.0',
 'c[18,300,16]1.0',
 'c[18,180,3]1.0',
 'c[19,240,14]1.0',
 'c[21,120,1]1.0',
 'c[22,90,2]1.0',
 'c[23,180,5]1.0',
 'c[24,180,11]1.0',
 'c[25,180,12]1.0',
 'c[25,120,4]1.0',
 'c[26,90,10]1.0',
 'c[27,120,9]1.0',
 'c[27,90,15]1.0',
 'c[28,90,13]1.0',
 'c[29,60,18]1.0',
 'c[30,120,8]1.0',
 'c[30,60,3]1.0',
 'c[32,90,10]1.0',
 'c[32,60,6]1.0',
 'c[33,90,18]1.0',
 'c[35,30,9]1.0',
 'c[35,30,14]1.0']

In [10]:
# Write to csv
with open('gs2_1.csv', 'w') as myfile:
   wr = csv.writer(myfile, quoting=csv.QUOTE_ALL,lineterminator='\n')
   wr.writerows(zip(var_names_start))

In [ ]:
chairlist = []
for time in v.keys():
    for chair,j in v[time].items():
        if j != 0:
            chairlist.append([time,chair,j])

In [ ]:
with open("chairout_3_8.csv","w",newline="") as f:
    writer = csv.writer(f)
    writer.writerows(chairlist)